**Een Identifier die gebruik maakt van active learning. 500 vragen zijn zelf gelabeled. Vervolgens fine tunen we een bert model hierop. We laten deze getunede bert dan zelf 1000 ongeziene vragen labelen. We kijken naar de 20% waar het model het minst zeker van is. Die kijken we handmatig na. De gecorrigeerde voegen we dan toe aan de training data en zo herhalen we dit proces tot het bert model naar behoren werkt.** 

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load labeled data
df = pd.read_excel("Trainig_data.xlsx")
df.columns = ['question', 'label']

# Remove duplicate questions
df = df.drop_duplicates(subset='question')

# Split into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['question'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [3]:
'''
Een random lange zin om te zien of we met 128 tokens per zin goed zitten. Deze zin bevat 44 dus 128 zou ok moeten zijn 
'''


tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')
tokenizer2 = BertTokenizer.from_pretrained("bert-base-multilingual-cased" )
tokenizer3 = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

text = "Zal de minister initiatieven nemen om ervoor te zorgen dat personen in een palliatief zorgtraject in de toekomst wel aanspraak kunnen maken op een tegemoetkoming van hulpmiddelen? Zo ja, welke en binnen welke termijn?"

tokens = tokenizer.tokenize(text)
tokens2 = tokenizer2.tokenize(text)
tokens3 = tokenizer3.tokenize(text)

print("Number of tokens:", len(tokens))
print("Tokens:", tokens)

print("Number of tokens:", len(tokens2))
print("Tokens:", tokens2)

print("Number of tokens:", len(tokens3))
print("Tokens:", tokens3)

Number of tokens: 44
Tokens: ['Zal', 'de', 'minister', 'initiatie', '##ven', 'nemen', 'om', 'ervoor', 'te', 'zorgen', 'dat', 'personen', 'in', 'een', 'pal', '##lia', '##tief', 'zorgt', '##ra', '##ject', 'in', 'de', 'toekomst', 'wel', 'aan', '##spraak', 'kunnen', 'maken', 'op', 'een', 'tegemoetkoming', 'van', 'hulpmiddel', '##en', '?', 'Zo', 'ja', ',', 'welke', 'en', 'binnen', 'welke', 'termijn', '?']
Number of tokens: 55
Tokens: ['Za', '##l', 'de', 'minister', 'init', '##iati', '##even', 'nemen', 'om', 'ervoor', 'te', 'zo', '##rgen', 'dat', 'personen', 'in', 'een', 'pal', '##lia', '##tief', 'zo', '##rgt', '##raj', '##ect', 'in', 'de', 'toe', '##komst', 'wel', 'aan', '##spraak', 'kunnen', 'maken', 'op', 'een', 'te', '##gem', '##oe', '##tkom', '##ing', 'van', 'hulp', '##mid', '##delen', '?', 'Zo', 'ja', ',', 'welke', 'en', 'binnen', 'welke', 'term', '##ijn', '?']
Number of tokens: 40
Tokens: ['Zal', 'Ġde', 'Ġminister', 'Ġinitiatieven', 'Ġnemen', 'Ġom', 'Ġervoor', 'Ġte', 'Ġzorgen', 'Ġdat'

**GroNLP**

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')  # or 'GroNLP/bert-base-dutch-cased'

def tokenize(texts):
    return tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt", max_length=128
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)


In [5]:

class QuestionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = QuestionDataset(train_encodings, train_labels)
val_dataset = QuestionDataset(val_encodings, val_labels)


In [6]:
model = BertForSequenceClassification.from_pretrained('GroNLP/bert-base-dutch-cased', num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.454600,0.272517,0.893671,0.890915,0.893671,0.891790
2,0.299200,0.370214,0.865823,0.884731,0.865823,0.871413
3,0.174600,0.482546,0.873418,0.884037,0.873418,0.877060
4,0.091500,0.608180,0.881013,0.893657,0.881013,0.884976
5,0.057100,0.618076,0.888608,0.890612,0.888608,0.889495
6,0.041100,0.632299,0.886076,0.893835,0.886076,0.888808
7,0.017600,0.665578,0.893671,0.891456,0.893671,0.892285
8,0.016400,0.659775,0.886076,0.887578,0.886076,0.886763


TrainOutput(global_step=1584, training_loss=0.14402549706324183, metrics={'train_runtime': 211.5651, 'train_samples_per_second': 59.67, 'train_steps_per_second': 7.487, 'total_flos': 830378490716160.0, 'train_loss': 0.14402549706324183, 'epoch': 8.0})

In [7]:
# Load test data
df_test = pd.read_excel("Testset.xlsx")
df_test.columns = ['question', 'label']

# Clean and convert to string
df_test = df_test.dropna(subset=['question', 'label'])
df_test['question'] = df_test['question'].astype(str)

# Extract questions and labels
test_texts = df_test['question'].tolist()
test_labels = df_test['label'].tolist()

# Tokenize
test_encodings = tokenizer(
    test_texts, padding=True, truncation=True, return_tensors="pt", max_length=128
)


In [8]:
from transformers import Trainer
import numpy as np
import pandas as pd

# Create the test dataset
test_dataset = QuestionDataset(test_encodings, test_labels)

# Run predictions
predictions_output = trainer.predict(test_dataset)

# Get raw logits
logits = predictions_output.predictions
predicted_labels = np.argmax(logits, axis=1)

# Load test questions (in same order)
df_test = pd.read_excel("Testset.xlsx")
df_test = df_test.dropna(subset=['question']).reset_index(drop=True)
df_test.columns = ['question', 'true_label']

# Create final DataFrame with question + true + predicted
df = pd.DataFrame({
    "question": df_test["question"],
    "true_label": df_test["true_label"],
    "predicted_label": predicted_labels
})

# Save
df.to_csv("bert_predictions.csv", index=False)




In [9]:
import numpy as np
from sklearn.metrics import classification_report

# Get raw predictions
outputs = trainer.predict(test_dataset)
preds = np.argmax(outputs.predictions, axis=1)

# Print evaluation report
print(classification_report(test_labels, preds))


              precision    recall  f1-score   support

           0       0.95      0.96      0.95       362
           1       0.90      0.85      0.88       137

    accuracy                           0.93       499
   macro avg       0.92      0.91      0.92       499
weighted avg       0.93      0.93      0.93       499



**mBERT**

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased" )  # or 'GroNLP/bert-base-dutch-cased'

def tokenize(texts):
    return tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt", max_length=128
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)


In [11]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.549000,0.488089,0.779747,0.828258,0.779747,0.683249
2,0.458000,0.395661,0.863291,0.883684,0.863291,0.838041
3,0.433100,0.459078,0.815190,0.812953,0.815190,0.814017
4,0.372900,0.362548,0.843038,0.862511,0.843038,0.849369
5,0.337300,0.401146,0.860759,0.866602,0.860759,0.863151
6,0.248100,0.604320,0.840506,0.863110,0.840506,0.847563
7,0.171400,0.571859,0.853165,0.868279,0.853165,0.858270
8,0.138900,0.507878,0.878481,0.876641,0.878481,0.877443


TrainOutput(global_step=1584, training_loss=0.33859208858374396, metrics={'train_runtime': 291.4924, 'train_samples_per_second': 43.308, 'train_steps_per_second': 5.434, 'total_flos': 830378490716160.0, 'train_loss': 0.33859208858374396, 'epoch': 8.0})

In [12]:
from transformers import Trainer
import numpy as np
import pandas as pd

# Create the test dataset
test_dataset = QuestionDataset(test_encodings, test_labels)

# Run predictions
predictions_output = trainer.predict(test_dataset)

# Get raw logits
logits = predictions_output.predictions
predicted_labels = np.argmax(logits, axis=1)

# Load test questions (in same order)
df_test = pd.read_excel("Testset.xlsx")
df_test = df_test.dropna(subset=['question']).reset_index(drop=True)
df_test.columns = ['question', 'true_label']

# Create final DataFrame with question + true + predicted
df = pd.DataFrame({
    "question": df_test["question"],
    "true_label": df_test["true_label"],
    "predicted_label": predicted_labels
})

# Save
df.to_csv("mbert_predictions.csv", index=False)

In [13]:
import numpy as np
from sklearn.metrics import classification_report

# Get raw predictions
outputs = trainer.predict(test_dataset)
preds = np.argmax(outputs.predictions, axis=1)

# Print evaluation report
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       362
           1       0.84      0.82      0.83       137

    accuracy                           0.91       499
   macro avg       0.89      0.88      0.89       499
weighted avg       0.91      0.91      0.91       499



**robBERT**

In [14]:
from transformers import BertTokenizer

tokenizer =RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base") # or 'GroNLP/bert-base-dutch-cased'

def tokenize(texts):
    return tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt", max_length=128
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)


In [15]:

model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.532400,0.522381,0.779747,0.828258,0.779747,0.683249
2,0.489400,0.462455,0.787342,0.750003,0.787342,0.743327
3,0.401900,0.382596,0.868354,0.867123,0.868354,0.853787
4,0.332000,0.408502,0.837975,0.839339,0.837975,0.838631
5,0.311200,0.638507,0.797468,0.826341,0.797468,0.807176
6,0.261700,0.489926,0.873418,0.867485,0.873418,0.867188
7,0.196000,0.572110,0.860759,0.863843,0.860759,0.862134
8,0.181900,0.552783,0.875949,0.870745,0.875949,0.871572


TrainOutput(global_step=1584, training_loss=0.3383156362206045, metrics={'train_runtime': 187.0494, 'train_samples_per_second': 67.49, 'train_steps_per_second': 8.468, 'total_flos': 830378490716160.0, 'train_loss': 0.3383156362206045, 'epoch': 8.0})

In [16]:
from transformers import Trainer
import numpy as np
import pandas as pd

# Create the test dataset
test_dataset = QuestionDataset(test_encodings, test_labels)

# Run predictions
predictions_output = trainer.predict(test_dataset)

# Get raw logits
logits = predictions_output.predictions
predicted_labels = np.argmax(logits, axis=1)

# Load test questions (in same order)
df_test = pd.read_excel("Testset.xlsx")
df_test = df_test.dropna(subset=['question']).reset_index(drop=True)
df_test.columns = ['question', 'true_label']

# Create final DataFrame with question + true + predicted
df = pd.DataFrame({
    "question": df_test["question"],
    "true_label": df_test["true_label"],
    "predicted_label": predicted_labels
})

# Save
df.to_csv("robbert_predictions.csv", index=False)

In [17]:
import numpy as np
from sklearn.metrics import classification_report

# Get raw predictions
outputs = trainer.predict(test_dataset)
preds = np.argmax(outputs.predictions, axis=1)

# Print evaluation report
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92       362
           1       0.84      0.72      0.77       137

    accuracy                           0.89       499
   macro avg       0.87      0.83      0.85       499
weighted avg       0.88      0.89      0.88       499



**regex**

In [18]:
import pandas as pd
import re
from sklearn.metrics import classification_report

# Load test data
df_test = pd.read_excel("Testset.xlsx")
df_test.columns = ['question', 'label']
df_test = df_test.dropna(subset=['question'])
df_test['question'] = df_test['question'].astype(str)

# Your regex sets
statistical_sets = {
    "Set 1": [
        r"\b(hoeveel|aantal|percentage van|percentage|cijfer over|data over|statistieken van)\b",
        r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van)?\s*(de|een)?\s*(gegevens|statistieken|cijfers)\b"
    ],
    # "Set 1.1": [
    #     r"\b(aantal|percentage van|percentage|cijfer over|data over|statistieken van)\b",
    #     r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
    #     r"\b(?:verschaffen|geven|tonen|lijst|overzicht van)?\s*(de|een)?\s*(gegevens|statistieken|cijfers)\b"
    # ],
    # "Set 2": [
    #     r"\b(hoe vaak|hoe groot|gemiddelde van|mediaan van|ratio van|procent van)\b",
    #     r"\b(stijging van|daling van|verandering in|ontwikkeling in|schommeling van|impact op)\b",
    #     r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|analyse van)?\s*(de|een)?\s*(cijfers|gegevens|statistieken|tellingen)\b"
    # ],
    # "Set 3": [
    #     r"\b(bruto|netto|inkomen|uitgaven|begroting|subsidies|belasting|tarief|BBP|inflatie|schulden|werkloosheid|bestedingen|consumptie)\b",
    #     r"\b(bevolking|leeftijdsgroep|demografie|migratie|emigratie|immigratie|huishoudens|gezinnen|verhouding tussen|dichtheid)\b",
    #     r"\b(aantal|hoeveelheid|grootte van|gemiddelde|mediaan|percentage|spreiding|percentiel|kwartiel|standaarddeviatie)\b"
    # ],
    # "Set 4": [
    #     r"\b(zorgkosten|patiënten|ziekenhuisopnames|sterftecijfers|levensverwachting|gezondheidsuitgaven|vaccinaties|epidemieën|medicatiegebruik)\b",
    #     r"\b(reistijd|filedruk|kilometers afgelegd|verkeersongevallen|CO2-uitstoot|luchtvervuiling|hernieuwbare energie|klimaatverandering|waterkwaliteit)\b",
    #     r"\b(grondprijzen|woningmarkt|huurprijzen|hypotheken|verkoopcijfers|bouwvergunningen|energieverbruik)\b"
    # ],
    # "Set 5": [
    #     r"\b(vergeleken met|ten opzichte van|in vergelijking met|in het verleden|sinds \d{4}|tussen \d{4} en \d{4})\b",
    #     r"\b(ontwikkeling sinds|historische gegevens|trendanalyse|jaarverslagen|statistische rapporten)\b"
    # ],
    # "Set 6": [
    #     r"\b(hoeveel|aantal|percentage|statistieken|cijfers|gegevens|data)\b",
    #     r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
    #     r"\b(per\s+\w+|per\s+\d+|in\s+\d{4}|tussen\s+\d{4}\s+en\s+\d{4})\b",  
    #     r"\b(wat is het aantal|hoe groot is|hoe vaak|gemiddelde van|ratio van)\b"
    # ],
    # "Set 7": [
    #     r"^(Wat is het aantal|Hoeveel)\b",
    #     r"\b(totale budget|welk budget|eenzelfde hoogte|gemiddeld aantal|geëvolueerd in de periode|jaarlijkse kostprijs |het aantal)\b",
    #     r"\b(bedroeg|welk|Kan de minister|wat was)\s*(de|een)?\s*(factuur|budget|overzicht|kostprijs)\b",
    #     r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|bedroeg)?\s*(de|een)?\s*(gegevens|statistieken|cijfers|factuur|overzicht)\b"
    # ],
    # "Set 8": [
    #     r"^(Wat is het aantal|Hoeveel)\b",
    #     r"\b(totale budget|welk budget|cijfer over|eenzelfde hoogte|gemiddeld aantal|geëvolueerd in de periode|jaarlijkse kostprijs|het aantal)\b",
    #     r"\b(bedroeg|welk|Kan de minister|wat was)\s*(de|een)?\s*(factuur|budget|overzicht|kostprijs)\b",
    #     r"\b(?:verschaffen|geven|tonen|lijst|overzicht van|bedroeg)?\s*(de|een)?\s*(gegevens|statistieken|cijfers|factuur|overzicht)\b"
    # ],
    "Set 2": [
        r"^(Hoeveel)\b"
    ]
}

# Function to apply regex rules to each question
def matches_any(question, patterns):
    return any(re.search(pat, question, flags=re.IGNORECASE) for pat in patterns)

# Evaluate each regex set
for set_name, patterns in statistical_sets.items():
    df_test[f"{set_name}_match"] = df_test['question'].apply(lambda q: int(matches_any(q, patterns)))

    if 'label' in df_test.columns:
        print(f"\n🧪 {set_name} — Evaluation against true labels:")
        print(classification_report(df_test['label'], df_test[f"{set_name}_match"], digits=3))



🧪 Set 1 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.914     0.914     0.914       362
           1      0.774     0.774     0.774       137

    accuracy                          0.876       499
   macro avg      0.844     0.844     0.844       499
weighted avg      0.876     0.876     0.876       499


🧪 Set 2 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.859     0.994     0.922       362
           1      0.975     0.569     0.719       137

    accuracy                          0.878       499
   macro avg      0.917     0.782     0.820       499
weighted avg      0.891     0.878     0.866       499



In [19]:
import pandas as pd
import re
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Load test data
df_test = pd.read_excel("Testset.xlsx")
df_test.columns = ['question', 'label']
df_test = df_test.dropna(subset=['question'])
df_test['question'] = df_test['question'].astype(str)

# Define regex sets
statistical_sets = {
    "Set_1": [
        r"\b(hoeveel|aantal|percentage van|percentage|cijfer over|data over|statistieken van)\b",
        r"\b(trend in|evolutie van|groei van|toename van|afname van|ontwikkeling van)\b",
        r"\b(?:verschaffen|geven|tonen|lijst|overzicht van)?\s*(de|een)?\s*(gegevens|statistieken|cijfers)\b"
    ],
    "Set_2": [
        r"^(Hoeveel)\b"
    ]
}

# Function to apply regex rules
def matches_any(question, patterns):
    return any(re.search(pat, question, flags=re.IGNORECASE) for pat in patterns)

# Save classification reports
metrics_list = []

# Apply and evaluate each regex set
for set_name, patterns in statistical_sets.items():
    col_name = f"{set_name}_match"
    df_test[col_name] = df_test['question'].apply(lambda q: int(matches_any(q, patterns)))

    if 'label' in df_test.columns:
        print(f"\n🧪 {set_name} — Evaluation against true labels:")
        report = classification_report(df_test['label'], df_test[col_name], digits=3, output_dict=True)
        print(classification_report(df_test['label'], df_test[col_name], digits=3))
        
        # Save key metrics
        prf = precision_recall_fscore_support(df_test['label'], df_test[col_name], average='binary')
        metrics_list.append({
            'Regex_Set': set_name,
            'Precision': prf[0],
            'Recall': prf[1],
            'F1-score': prf[2],
            'Support': prf[3]
        })

# Save predictions and metrics
df_test.to_csv("regex_predictions.csv", index=False)
pd.DataFrame(metrics_list).to_csv("regex_evaluation_metrics.csv", index=False)



🧪 Set_1 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.914     0.914     0.914       362
           1      0.774     0.774     0.774       137

    accuracy                          0.876       499
   macro avg      0.844     0.844     0.844       499
weighted avg      0.876     0.876     0.876       499


🧪 Set_2 — Evaluation against true labels:
              precision    recall  f1-score   support

           0      0.859     0.994     0.922       362
           1      0.975     0.569     0.719       137

    accuracy                          0.878       499
   macro avg      0.917     0.782     0.820       499
weighted avg      0.891     0.878     0.866       499



**Traditionals**

In [20]:
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords

# Make sure stopwords are downloaded
# import nltk; nltk.download('stopwords')
dutch_stopwords = stopwords.words('dutch')

# Load data
df_train = pd.read_excel("Trainig_data.xlsx")
df_test = pd.read_excel("Testset.xlsx")
df_train.columns = df_test.columns = ['question', 'label']
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# Vectorize
vectorizer = TfidfVectorizer(max_features=10000, stop_words=dutch_stopwords)
X_train = vectorizer.fit_transform(df_train['question'])
y_train = df_train['label']
X_test = vectorizer.transform(df_test['question'])
y_test = df_test['label']

# Store original test set
results_df = df_test.copy()

# Models
models = {
    "Naive_Bayes": MultinomialNB(alpha=0.1),
    "Logistic_Regression": LogisticRegression(
        C=10, class_weight='balanced', penalty='l2', solver='saga', max_iter=1000
    ),
    "SVM": SVC(C=10, class_weight='balanced'),
    "Random_Forest": RandomForestClassifier(
        class_weight='balanced', max_features='sqrt', n_estimators=300
    ),
    "XGBoost": XGBClassifier(
        max_depth=4, learning_rate=0.1, n_estimators=400,
        use_label_encoder=False, eval_metric='logloss'
    )
}

# Train, evaluate, and store predictions
for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    training_time = end_time - start_time

    preds = model.predict(X_test)
    results_df[name + '_pred'] = preds

    print(f"\n🧪 {name} (Training time: {training_time:.2f} seconds):\n")
    print(classification_report(y_test, preds, digits=3))

# Save predictions to CSV
results_df.to_csv("all_model_predictions.csv", index=False)





🧪 Naive_Bayes (Training time: 0.00 seconds):

              precision    recall  f1-score   support

           0      0.802     0.953     0.871       362
           1      0.754     0.380     0.505       137

    accuracy                          0.796       499
   macro avg      0.778     0.666     0.688       499
weighted avg      0.789     0.796     0.771       499


🧪 Logistic_Regression (Training time: 0.03 seconds):

              precision    recall  f1-score   support

           0      0.931     0.936     0.934       362
           1      0.830     0.818     0.824       137

    accuracy                          0.904       499
   macro avg      0.880     0.877     0.879       499
weighted avg      0.903     0.904     0.904       499


🧪 SVM (Training time: 0.38 seconds):

              precision    recall  f1-score   support

           0      0.896     0.956     0.925       362
           1      0.858     0.708     0.776       137

    accuracy                          0.8

C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\xgboost\core.py:158: UserWarning: [10:58:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🧪 XGBoost (Training time: 7.32 seconds):

              precision    recall  f1-score   support

           0      0.914     0.964     0.938       362
           1      0.889     0.759     0.819       137

    accuracy                          0.908       499
   macro avg      0.901     0.862     0.879       499
weighted avg      0.907     0.908     0.905       499



In [21]:
# # Load the files
# mbert_df = pd.read_csv("mbert_predictions.csv")
# robbert_df = pd.read_csv("robbert_predictions.csv")
# bert_df = pd.read_csv("bert_predictions.csv")
# models_df = pd.read_csv("all_model_predictions.csv")
# regex_df = pd.read_csv("regex_predictions.csv")

# # Add a temporary row index to preserve order
# mbert_df["row_index"] = range(len(mbert_df))
# robbert_df["row_index"] = range(len(robbert_df))
# bert_df["row_index"] = range(len(bert_df))
# models_df["row_index"] = range(len(models_df))
# regex_df["row_index"] = range(len(regex_df))

# # Standardize column names
# mbert_df = mbert_df.rename(columns={"true_label": "label"})
# robbert_df = robbert_df.rename(columns={"true_label": "label"})
# bert_df = bert_df.rename(columns={"true_label": "label"})
# models_df = models_df.rename(columns={"label": "label"})
# regex_df = regex_df.rename(columns={"label": "label"})

# # Merge on row_index (not question/label) to preserve order
# combined = pd.merge(bert_df, models_df.drop(columns=["question", "label"]), on="row_index")
# combined = pd.merge(combined, regex_df.drop(columns=["question", "label"]), on="row_index")

# # Drop row_index and reorder columns
# combined = combined.drop(columns=["row_index"])
# cols = ['question', 'label'] + [col for col in combined.columns if col not in ['question', 'label']]
# combined = combined[cols]

# # Save to file
# combined.to_csv("combined_predictions.csv", index=False)

In [22]:
import pandas as pd

# === 1. Load files fresh ===
mbert_df = pd.read_csv("mbert_predictions.csv")
robbert_df = pd.read_csv("robbert_predictions.csv")
bert_df = pd.read_csv("bert_predictions.csv")
models_df = pd.read_csv("all_model_predictions.csv")
regex_df = pd.read_csv("regex_predictions.csv")

# === 2. Remove any existing row_index columns, explicitly and thoroughly ===
for df_name, df in zip(["mbert_df", "robbert_df", "bert_df", "models_df", "regex_df"],
                       [mbert_df, robbert_df, bert_df, models_df, regex_df]):
    if "row_index" in df.columns:
        print(f"⚠️ {df_name} contains row_index, dropping it.")
        df.drop(columns=["row_index"], inplace=True)
    df["row_index"] = range(len(df))  # clean row_index added

# === 3. Rename predicted_label columns ===
mbert_df.rename(columns={"true_label": "label", "predicted_label": "mBERT_pred"}, inplace=True)
robbert_df.rename(columns={"true_label": "label", "predicted_label": "RobBERT_pred"}, inplace=True)
bert_df.rename(columns={"true_label": "label", "predicted_label": "GroNLP_pred"}, inplace=True)

# === 4. Merge all models step by step ===
combined = bert_df[["row_index", "question", "label", "GroNLP_pred"]]
combined = pd.merge(combined, mbert_df[["row_index", "mBERT_pred"]], on="row_index")
combined = pd.merge(combined, robbert_df[["row_index", "RobBERT_pred"]], on="row_index")

# Traditional model predictions
traditional_cols = [col for col in models_df.columns if col not in ["question", "label", "row_index"]]
combined = pd.merge(combined, models_df[["row_index"] + traditional_cols], on="row_index")

# Regex predictions
regex_cols = [col for col in regex_df.columns if col not in ["question", "label", "row_index"]]
combined = pd.merge(combined, regex_df[["row_index"] + regex_cols], on="row_index")

# === 5. Clean up and reorder ===
combined.drop(columns=["row_index"], inplace=True)
cols = ['question', 'label'] + [col for col in combined.columns if col not in ['question', 'label']]
combined = combined[cols]

# === 6. Save final merged file ===
combined.to_csv("combined_predictions.csv", index=False)
print("✅ Successfully saved to combined_predictions.csv")



✅ Successfully saved to combined_predictions.csv


In [23]:
print(models_df.columns)


Index(['question', 'label', 'Naive_Bayes_pred', 'Logistic_Regression_pred',
       'SVM_pred', 'Random_Forest_pred', 'XGBoost_pred', 'row_index'],
      dtype='object')


In [24]:
from scipy.stats import friedmanchisquare
import numpy as np
import pandas as pd

df = pd.read_csv("combined_predictions.csv")
models = [col for col in df.columns if col.endswith('_pred') or col.endswith('_match')]

# Binary correct predictions
correct_matrix = np.array([(df[model] == df['label']).astype(int) for model in models]).T

# Friedman test
stat, p = friedmanchisquare(*correct_matrix.T)
print(f"Friedman test statistic = {stat:.3f}, p-value = {p:.6f}")


Friedman test statistic = 122.442, p-value = 0.000000


In [25]:
import pandas as pd
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar
from itertools import combinations

# Load predictions
df = pd.read_csv("combined_predictions.csv")
models = [col for col in df.columns if col.endswith('_label') or col.endswith('_pred') or col.endswith('_match')]

# Ground truth
y_true = df['label'].values

# Prepare p-value result matrix
p_matrix = pd.DataFrame(index=models, columns=models, dtype=float)

# McNemar test function
def mcnemar_p(y_true, pred1, pred2):
    table = [[0, 0], [0, 0]]
    for y, a, b in zip(y_true, pred1, pred2):
        correct_a = int(a == y)
        correct_b = int(b == y)
        table[correct_b][correct_a] += 1
    result = mcnemar(table, exact=True)
    return result.pvalue

# Compare all model pairs
for model1, model2 in combinations(models, 2):
    p = mcnemar_p(y_true, df[model1], df[model2])
    p_matrix.loc[model1, model2] = p
    p_matrix.loc[model2, model1] = p  # symmetric

# Fill diagonal with 1s
np.fill_diagonal(p_matrix.values, 1.0)

# Print results
print("\n📊 McNemar's test p-values (lower = significant difference):")
print(p_matrix.round(4))




📊 McNemar's test p-values (lower = significant difference):
                          GroNLP_pred  mBERT_pred  RobBERT_pred  \
GroNLP_pred                    1.0000      0.0652        0.0002   
mBERT_pred                     0.0652      1.0000        0.1038   
RobBERT_pred                   0.0002      0.1038        1.0000   
Naive_Bayes_pred               0.0000      0.0000        0.0000   
Logistic_Regression_pred       0.0107      0.7428        0.2221   
SVM_pred                       0.0001      0.1263        1.0000   
Random_Forest_pred             0.0081      0.7493        0.1996   
XGBoost_pred                   0.0241      1.0000        0.1352   
Set_1_match                    0.0000      0.0270        0.5901   
Set_2_match                    0.0001      0.0402        0.6271   

                          Naive_Bayes_pred  Logistic_Regression_pred  \
GroNLP_pred                         0.0000                    0.0107   
mBERT_pred                          0.0000               

In [26]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import wilcoxon
from sklearn.metrics import classification_report

# Load predictions
df = pd.read_csv("combined_predictions.csv")
models = [col for col in df.columns if col.endswith('_label') or col.endswith('_pred') or col.endswith('_match')]

# Ground truth
y_true = df["label"].values

# Get per-class F1 scores
def get_f1_per_class(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
    return pd.Series(
        {label: metrics["f1-score"]
         for label, metrics in report.items()
         if label not in ["accuracy", "macro avg", "weighted avg"]}
    )

# Prepare matrix
p_matrix = pd.DataFrame(index=models, columns=models, dtype=float)

# Compare all model pairs
for model1, model2 in combinations(models, 2):
    f1_1 = get_f1_per_class(y_true, df[model1])
    f1_2 = get_f1_per_class(y_true, df[model2])

    # Ensure same class order
    f1_1, f1_2 = f1_1.align(f1_2, join='inner')

    try:
        stat, p = wilcoxon(f1_1, f1_2)
    except ValueError:
        # If samples are too small or identical
        p = np.nan

    p_matrix.loc[model1, model2] = p
    p_matrix.loc[model2, model1] = p

# Fill diagonal with 1s
np.fill_diagonal(p_matrix.values, 1.0)

# Print nicely
print("\n📊 Wilcoxon test p-values (lower = significant difference in per-class F1):")
print(p_matrix.round(4))



📊 Wilcoxon test p-values (lower = significant difference in per-class F1):
                          GroNLP_pred  mBERT_pred  RobBERT_pred  \
GroNLP_pred                       1.0         0.5           0.5   
mBERT_pred                        0.5         1.0           0.5   
RobBERT_pred                      0.5         0.5           1.0   
Naive_Bayes_pred                  0.5         0.5           0.5   
Logistic_Regression_pred          0.5         0.5           0.5   
SVM_pred                          0.5         0.5           0.5   
Random_Forest_pred                0.5         0.5           0.5   
XGBoost_pred                      0.5         1.0           0.5   
Set_1_match                       0.5         0.5           0.5   
Set_2_match                       0.5         0.5           0.5   

                          Naive_Bayes_pred  Logistic_Regression_pred  \
GroNLP_pred                            0.5                       0.5   
mBERT_pred                             0.5

In [27]:
def print_contingency(y_true, pred1, pred2, name1, name2):
    table = [[0, 0], [0, 0]]
    for y, a, b in zip(y_true, pred1, pred2):
        correct_a = int(a == y)
        correct_b = int(b == y)
        table[correct_b][correct_a] += 1
    print(f"\n{name1} vs {name2} contingency table (McNemar):")
    print(pd.DataFrame(table, columns=[f"{name2} correct", f"{name2} wrong"], 
                             index=[f"{name1} correct", f"{name1} wrong"]))


In [28]:
print_contingency(df['label'], df['predicted_label'], df['Logistic_Regression_pred'], 'BERT', 'LogReg')


KeyError: 'predicted_label'

In [ ]:
import pandas as pd

# Load predictions
df = pd.read_csv("combined_predictions.csv")

# Count correct predictions by BERT
bert_correct = (df['predicted_label'] == df['label']).sum()
total = len(df)

print(f"BERT predicted correctly on {bert_correct} out of {total} samples.")
print(f"Accuracy: {bert_correct / total:.3f}")


In [ ]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

# Load your combined predictions
df = pd.read_csv("combined_predictions.csv")

# Extract relevant predictions
y_true = df['label']
bert = df['predicted_label']
logreg = df['Logistic_Regression_pred']

# Count how many predictions each model got correct
bert_correct = (bert == y_true)
logreg_correct = (logreg == y_true)

print(f"BERT correct: {bert_correct.sum()} / {len(df)}")
print(f"LogReg correct: {logreg_correct.sum()} / {len(df)}")

# Build McNemar contingency table from scratch
# Rows: BERT (correct vs wrong)
# Cols: LogReg (correct vs wrong)
table = pd.crosstab(bert_correct, logreg_correct)
table.index = ['BERT wrong', 'BERT correct']
table.columns = ['LogReg wrong', 'LogReg correct']

print("\n✅ Contingency Table:")
print(table)

# Optional: McNemar p-value
print("\nMcNemar's test p-value:", mcnemar(table.values, exact=True).pvalue)


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def bootstrap_f1_diff(y_true, y_pred1, y_pred2, n=1000):
    diffs = []
    for _ in range(n):
        idx = np.random.choice(len(y_true), len(y_true), replace=True)
        f1_1 = f1_score(y_true[idx], y_pred1[idx])
        f1_2 = f1_score(y_true[idx], y_pred2[idx])
        diffs.append(f1_1 - f1_2)
    return np.mean(diffs), np.percentile(diffs, [2.5, 97.5])

# Example use
import pandas as pd
df = pd.read_csv("combined_predictions.csv")

y_true = df['label'].values
bert = df['predicted_label'].values
logreg = df['Logistic_Regression_pred'].values

mean_diff, ci = bootstrap_f1_diff(y_true, bert, logreg)

print(f"🔍 BERT vs Logistic Regression — F1 difference:")
print(f"Mean diff: {mean_diff:.4f}")
print(f"95% CI:    [{ci[0]:.4f}, {ci[1]:.4f}]")

